In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [2]:
bnf_1min = pd.read_csv("../data/gold_1d_tv.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2012]

In [3]:
bnf_1min.head()

,time,open,high,low,close,King Candle,Queen Candle,MA,datetime
0,2015-03-02,26600,26769,26500,26510,0,0,NaN,2015-03-02
1,2015-03-03,26500,26590,26380,26427,0,0,NaN,2015-03-03
2,2015-03-04,26411,26579,26409,26547,0,1,NaN,2015-03-04
3,2015-03-05,26565,26627,26435,26484,0,0,NaN,2015-03-05
4,2015-03-06,26460,26484,25982,26012,1,0,NaN,2015-03-06


In [4]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2016, 11, 7),
 datetime.date(2022, 8, 18),
 datetime.date(2023, 12, 6),
 datetime.date(2015, 6, 12),
 datetime.date(2022, 5, 2),
 datetime.date(2023, 5, 22),
 datetime.date(2023, 11, 8),
 datetime.date(2016, 12, 26),
 datetime.date(2021, 10, 5),
 datetime.date(2022, 8, 4),
 datetime.date(2017, 8, 8),
 datetime.date(2015, 3, 12),
 datetime.date(2018, 1, 1),
 datetime.date(2018, 1, 17),
 datetime.date(2024, 1, 8),
 datetime.date(2015, 6, 3),
 datetime.date(2017, 8, 21),
 datetime.date(2024, 3, 25),
 datetime.date(2023, 11, 28),
 datetime.date(2022, 2, 10),
 datetime.date(2016, 8, 19),
 datetime.date(2015, 8, 3),
 datetime.date(2021, 11, 15),
 datetime.date(2016, 8, 16),
 datetime.date(2021, 10, 12),
 datetime.date(2019, 5, 13),
 datetime.date(2022, 3, 21),
 datetime.date(2016, 10, 7),
 datetime.date(2021, 3, 2),
 datetime.date(2020, 12, 28),
 datetime.date(2021, 6, 16),
 datetime.date(2022, 8, 26),
 datetime.date(2024, 2, 21),
 datetime.date(2022, 6, 24),
 datetime.date(20

In [5]:
PORTFOLIO_VALUE = 10000000 # 10 Lacs
RPT_PCT = 0.05 # 5% RPT
SLIPPAGE_ = 0.001
LEVERAGE_ = 5
PYR_RPT_PCT = 0.01

In [6]:
data = bnf_1min

In [7]:
def calculate_signals(df, ma_length, ema_length):
    df['MA'] = df['close'].rolling(ma_length).mean()
    df['EMA'] = df['close'].ewm(span=ema_length, adjust=False).mean()
    
    # Signal conditions
    df['Buy_Signal'] = (
        (df['MA'] >= df['EMA']) & 
        (df['MA'].shift(1) < df['EMA'].shift(1))
    )
    return df

In [8]:
def backtest(df, fractal_num, sl_pct, pyr_num, pyr_rpt_pct):
    
    long_position = 0  # 0 = no position, 1 = long
    long_entry_price = 0
    long_entry_date = None
    long_trades = []
    tradebook = pd.DataFrame()
    # tradebook_long = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    long_trailing_stop = None
    tsl_time = None
    points = 0
    trade_number = 1
    pyramid_number = 0
    can_pyramid = True
    pyramid_high = 0

    for i in range(1, len(df)):

        if df.loc[i-1, 'Buy_Signal'] and long_position == 0:
            long_position = 1
            long_entry_price = df.loc[i-1, 'close']
            long_entry_date = df.loc[i-1, 'datetime']
            long_initial_sl = long_entry_price * (1 - (sl_pct) / 100)
            long_trailing_sl = long_initial_sl
            entry_remark = 'Long'
            # print('Long Found')
            # print(df.iloc[i])
            tradebook = pd.concat([tradebook, pd.DataFrame([{
                'Trade No.': trade_number,
                "Entry Date": long_entry_date.date(),
                "Entry Time": long_entry_date.time(),
                "Entry Price": long_entry_price,
                'Entry Remark': entry_remark,
                "Initial SL": long_initial_sl,
                'Final SL': long_trailing_sl,
                'TSL Time': None,
                "Exit Date": None,
                "Exit Time": None,
                "Exit Price": None,
                # "Points Captured": None,
                # "Slippages": None,
                # "Points w cs": None,
                # "PnL": None,
                "Remarks": None,
                # 'Qty': None,
                # "ROI%": None,
                "Trade Year": long_entry_date.year,
                "Trade Month": long_entry_date.month,
            }])], ignore_index=True)
            # can_add_long = False

        if long_position == 1:

            if all(df.loc[i - j, 'high'] <= df.loc[i - fractal_num, 'high'] for j in range(0, ((fractal_num * 2) + 1))):
                pyramid_high = df.loc[i - fractal_num, 'high']
                
            if all(df.loc[i - j, 'low'] >= df.loc[i - fractal_num, 'low'] for j in range(0, ((fractal_num * 2) + 1))):
                # Update Trailing SL
                long_trailing_sl = df.loc[i - fractal_num, 'low']
                tsl_time = df.loc[i - fractal_num, 'datetime']

            if df.loc[i, 'low'] <= long_initial_sl:
                # Initial SL Hit
                long_position = 0
                long_exit_price = long_initial_sl
                long_exit_time = df.loc[i, 'datetime']
                points = long_exit_price - long_entry_price
                remark = 'Initial SL'
                slippage = (long_entry_price + long_exit_price) * SLIPPAGE_
                qty = RPT_PCT * PORTFOLIO_VALUE / (long_entry_price - long_initial_sl) if entry_remark == 'Long' else pyr_rpt_pct * PORTFOLIO_VALUE / (long_entry_price - long_initial_sl)
                tradebook.loc[
                    (tradebook['Trade No.'] == trade_number),
                    [
                        'TSL Time', 
                        "Exit Date", 
                        "Exit Time",
                        "Exit Price",
                        # "Points Captured",
                        # "Slippages",
                        # "Points w cs",
                        # "PnL",
                        "Remarks",
                        # 'Qty',
                        # "ROI%",
                    ]
                ] = [
                    tsl_time,
                    long_exit_time.date(),
                    long_exit_time.time(),
                    long_exit_price,
                    # points,
                    # slippage,
                    # points - slippage,
                    # (points - slippage) * qty,
                    remark,
                    # qty,
                    # ((points - slippage) * qty / PORTFOLIO_VALUE) * 100,
                ]
                trade_number += 1
                can_pyramid = True
                pyramid_number = 0
                pyramid_high = 0

            elif df.loc[i, 'close'] <= long_trailing_sl:
                # Trailing SL Hit
                long_position = 0
                long_exit_price = df.loc[i, 'close']
                long_exit_time = df.loc[i, 'datetime']
                points = long_exit_price - long_entry_price
                remark = 'Trailing SL'
                slippage = (long_entry_price + long_exit_price) * SLIPPAGE_
                qty = RPT_PCT * PORTFOLIO_VALUE / (long_entry_price - long_initial_sl) if entry_remark == 'Long' else pyr_rpt_pct * PORTFOLIO_VALUE / (long_entry_price - long_initial_sl)
                tradebook.loc[
                    (tradebook['Trade No.'] == trade_number),
                    [
                        'TSL Time', 
                        "Exit Date", 
                        "Exit Time",
                        "Exit Price",
                        # "Points Captured",
                        # "Slippages",
                        # "Points w cs",
                        # "PnL",
                        "Remarks",
                        # 'Qty',
                        # "ROI%",
                    ]
                ] = [
                    tsl_time,
                    long_exit_time.date(),
                    long_exit_time.time(),
                    long_exit_price,
                    # points,
                    # slippage,
                    # points - slippage,
                    # (points - slippage) * qty,
                    remark,
                    # qty,
                    # ((points - slippage) * qty / PORTFOLIO_VALUE) * 100,
                ]
                trade_number += 1
                can_pyramid = True
                pyramid_number = 0
                pyramid_high = 0

            elif df.loc[i, 'MA'] < df.loc[i, 'EMA']:
                # MA Cross
                long_position = 0
                long_exit_price = df.loc[i, 'close']
                long_exit_time = df.loc[i, 'datetime']
                points = long_exit_price - long_entry_price
                remark = 'MA Bear Cross'
                slippage = (long_entry_price + long_exit_price) * SLIPPAGE_
                qty = RPT_PCT * PORTFOLIO_VALUE / (long_entry_price - long_initial_sl) if entry_remark == 'Long' else pyr_rpt_pct * PORTFOLIO_VALUE / (long_entry_price - long_initial_sl)
                tradebook.loc[
                    (tradebook['Trade No.'] == trade_number),
                    [
                        'TSL Time', 
                        "Exit Date", 
                        "Exit Time",
                        "Exit Price",
                        # "Points Captured",
                        # "Slippages",
                        # "Points w cs",
                        # "PnL",
                        "Remarks",
                        # 'Qty',
                        # "ROI%"
                    ]
                ] = [
                    tsl_time,
                    long_exit_time.date(),
                    long_exit_time.time(),
                    long_exit_price,
                    # points,
                    # slippage,
                    # points - slippage,
                    # (points - slippage) * qty,
                    remark,
                    # qty,
                    # ((points - slippage) * qty / PORTFOLIO_VALUE) * 100,
                ]
                trade_number += 1
                can_pyramid = True
                pyramid_number = 0
                pyramid_high = 0

            if can_pyramid and pyramid_number < pyr_num and pyramid_high > 0:
                if df.loc[i, 'high'] >= pyramid_high:
                    pyramid_entry_price = pyramid_high
                    pyramid_entry_date = df.loc[i, 'datetime']
                    pyramid_initial_sl = pyramid_entry_price * (1 - (sl_pct) / 100)
                    pyramid_trailing_sl = long_trailing_sl
                    entry_remark = 'Pyr Long'
                    tradebook = pd.concat([tradebook, pd.DataFrame([{
                        'Trade No.': trade_number,
                        "Entry Date": pyramid_entry_date.date(),
                        "Entry Time": pyramid_entry_date.time(),
                        "Entry Price": pyramid_entry_price,
                        'Entry Remark': entry_remark,
                        "Initial SL": pyramid_initial_sl,
                        'Final SL': pyramid_trailing_sl,
                        'TSL Time': None,
                        "Exit Date": None,
                        "Exit Time": None,
                        "Exit Price": None,
                        # "Points Captured": None,
                        # "Slippages": None,
                        # "Points w cs": None,
                        # "PnL": None,
                        "Remarks": None,
                        # 'Qty': None,
                        # "ROI%": None,
                        "Trade Year": long_entry_date.year,
                        "Trade Month": long_entry_date.month,
                    }])], ignore_index=True)
                    pyramid_number += 1
                    pyramid_high = 0
    

        # if points:
        #     slippage = (long_entry_price + long_exit_price) * SLIPPAGE_
        #     qty = RPT_PCT * PORTFOLIO_VALUE / (long_entry_price - long_initial_sl)
        #     trade = {
        #         "Entry Date": long_entry_date.date(),
        #         "Entry Time": long_entry_date.time(),
        #         "Entry Price": long_entry_price,
        #         "Initial SL": long_initial_sl,
        #         'Final SL': long_trailing_sl,
        #         'TSL Time': tsl_time,
        #         "Exit Date": long_exit_time.date(),
        #         "Exit Time": long_exit_time.time(),
        #         "Exit Price": long_exit_price,
        #         "Points Captured": points,
        #         "Slippages": slippage,
        #         "Points w cs": points - slippage,
        #         "PnL": (points - slippage) * qty,
        #         "Remarks": remark,
        #         'Qty': qty,
        #         "ROI%": ((points - slippage) * qty / PORTFOLIO_VALUE) * 100,
        #         "Trade Year": long_entry_date.year,
        #         "Trade Month": long_entry_date.month,
        #     }
        #     # print(trade)
        #     long_trades.append(trade)
        #     points = 0
        #     remark = ''

    # tradebook = pd.DataFrame(long_trades)
    return tradebook

In [9]:
MA_LENGTH = 14
EMA_LENGTH = 45
FRACTAL_LENGTH = 4
SL_PCT = 2
PYR_NUM = 1
pyr_rpt_pct = 0.01
# pyr_rpt_pct = [0.01, 0.02, 0.03, 0.04, 0.05]

df = calculate_signals(data, MA_LENGTH, EMA_LENGTH)
tb = backtest(df, FRACTAL_LENGTH, SL_PCT, PYR_NUM, pyr_rpt_pct)

In [10]:
# "Points Captured": None,
# "Slippages": None,
# "Points w cs": None,
# "PnL": None,
# "Remarks": None,
# 'Qty': None,
# "ROI%": None,

tb['Points Captured'] = tb['Exit Price'] - tb['Entry Price']
tb['Slippage'] = (tb['Entry Price'] + tb['Exit Price']) * SLIPPAGE_
tb['Points w cs'] = tb['Points Captured'] - tb['Slippage']
tb['Qty'] = np.where(
    tb['Entry Remark'] == 'Long',
    RPT_PCT * PORTFOLIO_VALUE / (tb['Entry Price'] - tb['Initial SL']),
    PYR_RPT_PCT * PORTFOLIO_VALUE / (tb['Entry Price'] - tb['Initial SL'])
)
tb['PnL'] = tb['Points w cs'] * tb['Qty']
tb['ROI%'] = tb['PnL'] * 100 / PORTFOLIO_VALUE
# tb

In [11]:
tb['Points Captured'].sum() , tb['Slippage'].sum() , tb['Points w cs'].sum()

(38301.92, 4142.481920000001, 34159.43807999999)

In [12]:
# tb.to_csv('TB PYR.csv')

In [13]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2015, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2015, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [14]:
stats = generate_stats(tb, '...')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

/tmp/ipykernel_4909/61047136.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2015,-10.7290,4,0.0000,NaN,-2.6823,-10.4056,-1.0311,...
2016,28.4207,5,40.0000,15.7238,-1.0090,-1.6000,17.7626,...
2017,-1.0668,8,37.5000,3.2726,-2.1769,-8.5521,-0.1247,...
2018,18.1831,5,80.0000,5.0189,-1.8923,-1.8923,9.6089,...
2019,44.9706,5,60.0000,15.5815,-0.8870,-1.1151,40.3277,...
2020,0.7683,2,50.0000,6.2633,-5.4950,-5.4950,0.1398,...
2021,-4.2664,7,42.8571,2.8861,-3.2312,-5.4950,-0.7764,...
2022,32.1811,6,66.6667,10.7928,-5.4950,-10.9900,2.9282,...
2023,7.6640,6,66.6667,3.0906,-2.3491,-4.6983,1.6312,...
2024,28.8650,4,50.0000,14.4325,NaN,0.0000,inf,...


# Gold 1D W Fractal Trailing

In [15]:
tb.to_csv('gold_1d_w_fractal_trailing.csv')

In [ ]:
# MA_LENGTH = 9
# EMA_LENGTH = 30
# FRACTAL_LENGTH = 5
# SL_PCT = 1

# df = calculate_signals(data, MA_LENGTH, EMA_LENGTH)
# tb = backtest(df, FRACTAL_LENGTH, SL_PCT)

sl_range = [0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5]
pyr_num_list = [1, 2, 3, 4, 5, 6, 7]
pyr_rpt_pct_list = [0.01, 0.02, 0.03, 0.04, 0.05]
stats_dictionary = {}

for i in range(8, 33, 2):
    for j in range(5, 81, 5):
        for k in range(1, 6):
            for sl in sl_range:
                for pyr_num in pyr_num_list:
                    for pyr_rpt_pct in pyr_rpt_pct_list:
                        if i < j and (j - i) >= 6:
                            df = calculate_signals(data, i, j)
                            tb = backtest(df, k, sl, pyr_num, pyr_rpt_pct)
                            variation = f'{i}, {j}, {k}, {sl}, {pyr_num}, {pyr_rpt_pct}'
                            print(variation)
                            # print(tb.to_string())
                            tb['Points Captured'] = tb['Exit Price'] - tb['Entry Price']
                            tb['Slippage'] = (tb['Entry Price'] + tb['Exit Price']) * SLIPPAGE_
                            tb['Points w cs'] = tb['Points Captured'] - tb['Slippage']
                            tb['Qty'] = np.where(
                                tb['Entry Remark'] == 'Long',
                                RPT_PCT * PORTFOLIO_VALUE / (tb['Entry Price'] - tb['Initial SL']),
                                PYR_RPT_PCT * PORTFOLIO_VALUE / (tb['Entry Price'] - tb['Initial SL'])
                            )
                            tb['PnL'] = tb['Points w cs'] * tb['Qty']
                            tb['ROI%'] = tb['PnL'] * 100 / PORTFOLIO_VALUE
                            stats = generate_stats(tb, variation)
                            for x, y in stats.items():
                                if x > 10 and len(tb) > 0:
                                    print(y.to_string())
                                    stats_dictionary[x] = y